Set up Spark

1. Extract

In [25]:
# this was how i was able to use pyspark i cannot get it to work when i install apache spark onto my mac - jenny
# activate .venv environment
# windows: .\venv\Scripts\activate
# mac: source .venv/bin/activate
# then pip install kaggle, pip install pyspark and pip install findspark inside .venv
# move kaggle.json file into ~/.kaggle
# then you can run data

import kaggle
from pyspark.sql import SparkSession
import findspark
import pandas as pd
findspark.init()
findspark.find()

dataset = "rohanrao/formula-1-world-championship-1950-2020"

kaggle.api.dataset_download_files(dataset, path='./', unzip=True)

print(f"Dataset {dataset} downloaded successfully!")

 # create spark session
spark = SparkSession.builder.master("local").appName("Driver_Performance_Prediction").getOrCreate()

Dataset URL: https://www.kaggle.com/datasets/rohanrao/formula-1-world-championship-1950-2020
Dataset rohanrao/formula-1-world-championship-1950-2020 downloaded successfully!


In [26]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType
from pyspark.sql.types import DoubleType

results = "results.csv"
qualifying = "qualifying.csv"
lap_times = "lap_times.csv"
pit_stops = "pit_stops.csv"
driver_standings = "driver_standings.csv"
races = "races.csv"
constructors = "constructors.csv"
status = "status.csv"

2. Transform

In [27]:
result_df = spark.read.csv(results, header=True, inferSchema=True)

# need to remove \N
result_df = result_df.dropna()
result_df = result_df.drop('number', 'positionText', 'time', 'rank','fastestLap', 'fastestLapTime','fastestLapSpeed', 'time', 'laps', 'points', 'positionOrder', 'grid')
result_df = result_df.withColumn('position', col('position').cast(IntegerType()))
result_df = result_df.withColumn('milliseconds', col('milliseconds').cast(IntegerType()))
result_df = result_df.dropna()
result_df = result_df.withColumnRenamed('position', 'final_position')

result_df.show()
print(result_df.dtypes)

+--------+------+--------+-------------+--------------+------------+--------+
|resultId|raceId|driverId|constructorId|final_position|milliseconds|statusId|
+--------+------+--------+-------------+--------------+------------+--------+
|       1|    18|       1|            1|             1|     5690616|       1|
|       2|    18|       2|            2|             2|     5696094|       1|
|       3|    18|       3|            3|             3|     5698779|       1|
|       4|    18|       4|            4|             4|     5707797|       1|
|       5|    18|       5|            1|             5|     5708630|       1|
|      23|    19|       8|            6|             1|     5478555|       1|
|      24|    19|       9|            2|             2|     5498125|       1|
|      25|    19|       5|            1|             3|     5517005|       1|
|      26|    19|      15|            7|             4|     5524387|       1|
|      27|    19|       1|            1|             5|     5525

In [28]:
qualifying_df = spark.read.csv(qualifying, header=True, inferSchema=True)
qualifying_df = qualifying_df.drop('number', 'q1', 'q2', 'q3', 'qualifyId')
qualifying_df = qualifying_df.withColumnRenamed('position', 'qualifying_position').withColumnRenamed('constructorId', 'qualifying_constructorId')
qualifying_df.show()
print(qualifying_df.dtypes)

+------+--------+------------------------+-------------------+
|raceId|driverId|qualifying_constructorId|qualifying_position|
+------+--------+------------------------+-------------------+
|    18|       1|                       1|                  1|
|    18|       9|                       2|                  2|
|    18|       5|                       1|                  3|
|    18|      13|                       6|                  4|
|    18|       2|                       2|                  5|
|    18|      15|                       7|                  6|
|    18|       3|                       3|                  7|
|    18|      14|                       9|                  8|
|    18|      10|                       7|                  9|
|    18|      20|                       5|                 10|
|    18|      22|                      11|                 11|
|    18|       4|                       4|                 12|
|    18|      18|                      11|             

In [29]:
lap_times_df = spark.read.csv(lap_times, header=True, inferSchema=True)
lap_times_df = lap_times_df.drop('time', 'lap', 'position')
lap_times_df = lap_times_df.withColumnRenamed('milliseconds', 'lap_time_milliseconds')
lap_times_df.show()
print(lap_times_df.dtypes)

+------+--------+---------------------+
|raceId|driverId|lap_time_milliseconds|
+------+--------+---------------------+
|   841|      20|                98109|
|   841|      20|                93006|
|   841|      20|                92713|
|   841|      20|                92803|
|   841|      20|                92342|
|   841|      20|                92605|
|   841|      20|                92502|
|   841|      20|                92537|
|   841|      20|                93240|
|   841|      20|                92572|
|   841|      20|                92669|
|   841|      20|                92902|
|   841|      20|                93698|
|   841|      20|               112075|
|   841|      20|                98385|
|   841|      20|                91548|
|   841|      20|                90800|
|   841|      20|                91810|
|   841|      20|                91018|
|   841|      20|                91055|
+------+--------+---------------------+
only showing top 20 rows

[('raceId', 'i

In [30]:
pit_stops_df = spark.read.csv(pit_stops, header=True, inferSchema=True)
pit_stops_df = pit_stops_df.drop('milliseconds', 'time', 'lap')
pit_stops_df = pit_stops_df.withColumnRenamed('duration', 'stop_duration')
pit_stops_df = pit_stops_df.withColumnRenamed('stop', 'pit_stop')
pit_stops_df = pit_stops_df.withColumn('stop_duration', col('stop_duration').cast(DoubleType()))
pit_stops_df.show()
print(pit_stops_df.dtypes)

+------+--------+--------+-------------+
|raceId|driverId|pit_stop|stop_duration|
+------+--------+--------+-------------+
|   841|     153|       1|       26.898|
|   841|      30|       1|       25.021|
|   841|      17|       1|       23.426|
|   841|       4|       1|       23.251|
|   841|      13|       1|       23.842|
|   841|      22|       1|       23.643|
|   841|      20|       1|       22.603|
|   841|     814|       1|       24.863|
|   841|     816|       1|       25.259|
|   841|      67|       1|       25.342|
|   841|       2|       1|       22.994|
|   841|       1|       1|       23.227|
|   841|     808|       1|       24.535|
|   841|       3|       1|       23.716|
|   841|     155|       1|       24.064|
|   841|      16|       1|       25.978|
|   841|      15|       1|       24.899|
|   841|      18|       1|       16.867|
|   841|     153|       2|       24.463|
|   841|       5|       1|       24.865|
+------+--------+--------+-------------+
only showing top

In [31]:
# driver position is driver's championship standing
driver_standings_df = spark.read.csv(driver_standings, header=True, inferSchema=True)
driver_standings_df = driver_standings_df.drop('position','positionText', 'wins', 'driverStandingsId')
driver_standings_df = driver_standings_df.withColumnRenamed('points', 'driver_points')
driver_standings_df.show()
print(driver_standings_df.dtypes)

+------+--------+-------------+
|raceId|driverId|driver_points|
+------+--------+-------------+
|    18|       1|         10.0|
|    18|       2|          8.0|
|    18|       3|          6.0|
|    18|       4|          5.0|
|    18|       5|          4.0|
|    18|       6|          3.0|
|    18|       7|          2.0|
|    18|       8|          1.0|
|    19|       1|         14.0|
|    19|       2|         11.0|
|    19|       3|          6.0|
|    19|       4|          6.0|
|    19|       5|         10.0|
|    19|       6|          3.0|
|    19|       7|          2.0|
|    19|       8|         11.0|
|    19|       9|          8.0|
|    19|      15|          5.0|
|    19|      17|          2.0|
|    19|      14|          0.0|
+------+--------+-------------+
only showing top 20 rows

[('raceId', 'int'), ('driverId', 'int'), ('driver_points', 'double')]


In [32]:
races_df = spark.read.csv(races, header=True, inferSchema=True)
races_df = races_df.drop('date','time', 'round','url', 'fp1_date', 'fp1_time', 'fp2_date', 'fp2_time', 'fp3_date', 'fp3_time', 'quali_date', 'quali_time', 'sprint_date', 'sprint_time')
races_df = races_df.withColumnRenamed('name', 'circuit_name')
races_df.show()
print(races_df.dtypes)

+------+----+---------+--------------------+
|raceId|year|circuitId|        circuit_name|
+------+----+---------+--------------------+
|     1|2009|        1|Australian Grand ...|
|     2|2009|        2|Malaysian Grand Prix|
|     3|2009|       17|  Chinese Grand Prix|
|     4|2009|        3|  Bahrain Grand Prix|
|     5|2009|        4|  Spanish Grand Prix|
|     6|2009|        6|   Monaco Grand Prix|
|     7|2009|        5|  Turkish Grand Prix|
|     8|2009|        9|  British Grand Prix|
|     9|2009|       20|   German Grand Prix|
|    10|2009|       11|Hungarian Grand Prix|
|    11|2009|       12| European Grand Prix|
|    12|2009|       13|  Belgian Grand Prix|
|    13|2009|       14|  Italian Grand Prix|
|    14|2009|       15|Singapore Grand Prix|
|    15|2009|       22| Japanese Grand Prix|
|    16|2009|       18|Brazilian Grand Prix|
|    17|2009|       24|Abu Dhabi Grand Prix|
|    18|2008|        1|Australian Grand ...|
|    19|2008|        2|Malaysian Grand Prix|
|    20|20

In [33]:
constructors_df = spark.read.csv(constructors, header=True, inferSchema=True)
constructors_df = constructors_df.drop('url', 'nationality', 'constructorRef')
constructors_df = constructors_df.withColumnRenamed('name', 'constructor_name')
constructors_df.show()
print(constructors_df.dtypes)

+-------------+----------------+
|constructorId|constructor_name|
+-------------+----------------+
|            1|         McLaren|
|            2|      BMW Sauber|
|            3|        Williams|
|            4|         Renault|
|            5|      Toro Rosso|
|            6|         Ferrari|
|            7|          Toyota|
|            8|     Super Aguri|
|            9|        Red Bull|
|           10|     Force India|
|           11|           Honda|
|           12|          Spyker|
|           13|             MF1|
|           14|      Spyker MF1|
|           15|          Sauber|
|           16|             BAR|
|           17|          Jordan|
|           18|         Minardi|
|           19|          Jaguar|
|           20|           Prost|
+-------------+----------------+
only showing top 20 rows

[('constructorId', 'int'), ('constructor_name', 'string')]


In [34]:
status_df = spark.read.csv(status, header=True, inferSchema=True)
status_df.show()
print(status_df.dtypes)

+--------+------------+
|statusId|      status|
+--------+------------+
|       1|    Finished|
|       2|Disqualified|
|       3|    Accident|
|       4|   Collision|
|       5|      Engine|
|       6|     Gearbox|
|       7|Transmission|
|       8|      Clutch|
|       9|  Hydraulics|
|      10|  Electrical|
|      11|      +1 Lap|
|      12|     +2 Laps|
|      13|     +3 Laps|
|      14|     +4 Laps|
|      15|     +5 Laps|
|      16|     +6 Laps|
|      17|     +7 Laps|
|      18|     +8 Laps|
|      19|     +9 Laps|
|      20|    Spun off|
+--------+------------+
only showing top 20 rows

[('statusId', 'int'), ('status', 'string')]


In [37]:
from pyspark.sql.functions import isnan, when, count

# merge datasets together
merge1_df = result_df.join(races_df, "raceId", "left")
merge2_df = merge1_df.join(constructors_df, "constructorId", "left")
merge3_df = merge2_df.join(driver_standings_df, ["driverId", "raceId"], "left")
merge4_df = merge3_df.join(lap_times_df, ["raceId","driverId"], "left")
merge5_df = merge4_df.join(pit_stops_df, ["raceId","driverId"], "left")
merge6_df = merge5_df.join(qualifying_df, ["raceId","driverId"], "left")
combined_df = merge6_df.join(status_df, "statusId", "left")

combined_df.select('year').distinct().collect()

#combined_df.drop(combined_df.qualifying_constructorId).show()

# filter out to show only drivers who have finished a race
#combined_df.filter(combined_df.status == "Finished")

# show all nan/nulls
#combined_df.select([count(when(isnan(c)| col(c).isNull(), c)).alias(c) for c in combined_df.columns]).show()

# drop nulls
#combined_df.dropna().show(5)

# double check
#combined_df.select([count(when(isnan(c), c)).alias(c) for c in combined_df.columns]).show()

+--------+------+--------+-------------+--------+--------------+------------+----+---------+--------------------+----------------+-------------+---------------------+--------+-------------+------------------------+-------------------+--------+
|statusId|raceId|driverId|constructorId|resultId|final_position|milliseconds|year|circuitId|        circuit_name|constructor_name|driver_points|lap_time_milliseconds|pit_stop|stop_duration|qualifying_constructorId|qualifying_position|  status|
+--------+------+--------+-------------+--------+--------------+------------+----+---------+--------------------+----------------+-------------+---------------------+--------+-------------+------------------------+-------------------+--------+
|       1|   841|      20|            9|   20779|             1|     5370259|2011|        1|Australian Grand ...|        Red Bull|         25.0|                91946|       2|       24.036|                       9|                  1|Finished|
|       1|   841|      2

24/12/17 01:35:09 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 195556 ms exceeds timeout 120000 ms
24/12/17 01:35:09 WARN SparkContext: Killing executors is not supported by current scheduler.
24/12/17 01:35:12 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at o

3. EDA

In [36]:
import pandas as pd

# i dropped the nan/nulls in pyspark but for some reason it doesnt transfer over here + filtering
df = combined_df.toPandas()

df = df.dropna() 

filtered_df = df[df['status'] == 'Finished']
print(filtered_df['status'].unique())
#filtered_df = df[df['year'] > 1980]
a = filtered_df['year'].unique()
a.sort()
print(a)

filtered_df = filtered_df.drop('qualifying_constructorId', axis=1)

filtered_df.head()


['Finished']
[2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023 2024]


,statusId,raceId,driverId,constructorId,resultId,final_position,milliseconds,year,circuitId,circuit_name,constructor_name,driver_points,lap_time_milliseconds,pit_stop,stop_duration,qualifying_position,status
125528,1,841,20,9,20779,1,5370259,2011,1,Australian Grand Prix,Red Bull,25.0,91946.0,2.0,24.036,1.0,Finished
125529,1,841,20,9,20779,1,5370259,2011,1,Australian Grand Prix,Red Bull,25.0,91946.0,1.0,22.603,1.0,Finished
125530,1,841,20,9,20779,1,5370259,2011,1,Australian Grand Prix,Red Bull,25.0,90516.0,2.0,24.036,1.0,Finished
125531,1,841,20,9,20779,1,5370259,2011,1,Australian Grand Prix,Red Bull,25.0,90516.0,1.0,22.603,1.0,Finished
125532,1,841,20,9,20779,1,5370259,2011,1,Australian Grand Prix,Red Bull,25.0,89962.0,2.0,24.036,1.0,Finished
